In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import re
from keras.preprocessing import sequence
from flask import Flask, jsonify, request,render_template


# https://www.tutorialspoint.com/flask
import flask
app = Flask(__name__)

#loading of keyword
Question_keywords = np.load(r'Questin_Answer_10000_keywords/Question_keywords.npy',allow_pickle = True)
Answer_keywords = np.load(r'Questin_Answer_10000_keywords/Answer_keywords.npy',allow_pickle = True)

#load config & weight of best model
with open('Best_model/config_9.p', 'rb') as fp:
	config = pickle.load(fp)

with open('Best_model/Best_weights_9.p', 'rb') as fp:
	weights = pickle.load(fp)

#loading weights to model
model = tf.keras.Model.from_config(config)
model.set_weights(weights)

def final_fun_1(X):
    '''Takes in raw data do pre processing and returnes predicted result 0 indicates not correct answer,1 indicates correct answer. each question can have only cone correct answer'''
    data = X.copy()
    preprocessed_Questions = []
	
	#removel of spcial Charaters
    for sentance in (data['Question'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Questions.append(sent)
    
    preprocessed_Answers = []
    for sentance in (data['Answer'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Answers.append(sent)
        
    data['Question'] = preprocessed_Questions
    data['Answer'] = preprocessed_Answers
    
    #removel of non keywords
    que_which_has_ly_keywords = []
    for sentance in data['Question']:
        sent = ' '.join(e for e in sentance.split() if e in Question_keywords)
        que_which_has_ly_keywords.append(sent)
    
     
    ans_which_has_ly_keywords = []
    for sentance in data['Answer']:
        sent = ' '.join(e for e in sentance.split() if e in Answer_keywords)
        ans_which_has_ly_keywords.append(sent)
        
    data['Question'] = que_which_has_ly_keywords
    data['Answer'] = ans_which_has_ly_keywords
        
        
    #Setting numerical value to each word in questions
    ecoded_ques = []
    for train_ques in data['Question']:
        ecoded_ques.append(list(map(lambda x: list(Question_keywords).index(x)+1,train_ques.split(" "))))
    
    ecoded_ans = []
    for train_ans in data['Answer']:
        ecoded_ans.append(list(map(lambda x: list(Answer_keywords).index(x)+1,train_ans.split(" "))))
    
    data['Question_vector'] = ecoded_ques
    data['Answer_vector'] = ecoded_ans
    
    
    #truncate and/or pad input sequences
    Max_Question_length = 24
    ques_vect_1 = sequence.pad_sequences(list(data.Question_vector), maxlen=Max_Question_length)
    ques_vect = [np.hstack((x,10001)) for x in ques_vect_1]
        
    # truncate and/or pad input sequences
    Max_Answer_length = 163
    ans_vect = sequence.pad_sequences(data.Answer_vector, maxlen=Max_Answer_length)
    
    
    ques_vect_ds = tf.data.Dataset.from_tensor_slices((ques_vect)).batch(1024)
    ans_vect_ds = tf.data.Dataset.from_tensor_slices((ans_vect)).batch(1024)
    

    @tf.function
    def predict(input_vec):
        '''Taks in a batch of vector ,pass it into model and returns the predicted value'''
        predictions = model(input_vec, training=False)
        return predictions

    #stack the output of each batch
    def predicted_y(ques_vect,ans_vect):
        '''Takes in the a two vector(Question,answe vecror) and returns the predicted result'''
        ls = []
        for question_batch,ans_batch in zip(ques_vect_ds,ans_vect_ds):
            test_ques_vec = question_batch
            test_ans_vec = ans_batch
            ls.append(predict([test_ques_vec,test_ans_vec]))

        pred_y = ls[0]
        for i in range(1,len(ls)):
            pred_y = np.vstack((pred_y,ls[i]))
        return pred_y
    
    predicted_y = predicted_y(ques_vect_ds,ans_vect_ds)
    
    for_output = np.zeros_like(predicted_y,dtype = np.int32)
    for i in range(0,predicted_y.shape[0],10):
        maxi = np.argmax(predicted_y[i:i+10])
        for_output[i+maxi] = 1
        

    return np.array(for_output[:,0])




@app.route('/', methods=['GET','POST'])
def index():
    return render_template('index.html')

@app.route('/data', methods=['POST'])
def data():
    if request.method == 'POST':
        file = request.form['upload-file']
        data = pd.read_csv(file)
        data['Result']= final_fun_1(data)


        return render_template('data.html',total_data = data.loc[:,['Question','Answer','Respose_number']].to_html() ,data=data[data['Result']==1].loc[:,['Question','Answer','Respose_number']].to_html())


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
